# 準備

分析やアプリケーションでの利用を考える前に不要なものを取り除く  
ツイートは令和4年8月にTwitter上で流通した"デマ　OR ガセ OR 嘘 OR ウソ OR フェイク"で収集した  
最後は集計と可視化の前には前処理が必要なことを示す

In [1]:
# コードの自動整形、読み込み、進捗の可視化を追加
! pip install jupyter-black pandas tqdm

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
# コード自動整形の有効化
import jupyter_black

jupyter_black.load()

In [3]:
# データの読み込み
import pandas
import pathlib
from tqdm.notebook import tqdm

# チュートリアルのデータがある場所を指定する
dir_path = pathlib.Path("/content/drive/MyDrive/tutorial_2nd_データクリーニング/parsed")
pattern = "*.jsonl"
file_path_list = list(dir_path.glob(pattern))

# 10分間隔で保存されているので
buff = list()
for file_path in tqdm(file_path_list):
    # created_atの情報をつかわないなら、JSTの情報は消さなくてもOK（tutorial1参照）
    df = pandas.read_json(file_path, lines=True, orient="record")
    buff.append(df)

df = pandas.concat(buff)

  0%|          | 0/1007 [00:00<?, ?it/s]

In [4]:
# 中身の確認。RTが多い。
df

,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
0,2022-08-22 00:00:00+09:00,1561367519503323136,RT @Narodovlastiye: 年間所得200万円程度と、決して生活に余裕のない高齢...,1034458998861258752,Twitter for iPhone,Daydreamer,HeadinCloudNine,0,0,8065,0,True,False,False,ja,[],https://twitter.com/1034458998861258753/status...,RT @USER_NAME: 年間所得200万円程度と、決して生活に余裕のない高齢者に対して...
1,2022-08-22 00:00:00+09:00,1561367521172697088,RT @fmb3l9isWPTEBDy: 年間所得200万円程度と、決して生活に余裕のない高...,1246062549151318016,feather for iOS,ゆん☺︎,by_mnma,0,0,3883,0,True,False,False,ja,[],https://twitter.com/1246062549151318019/status...,RT @USER_NAME: 年間所得200万円程度と、決して生活に余裕のない高齢者に対して...
2,2022-08-22 00:00:00+09:00,1561367521294372864,RT @crx7601: 韓国紙「日本経済を抜くと言ったな？あれは嘘だ、それどころかフィリピ...,922613564,Twitter for Android,mighty-ko,mightyguy1357,0,0,7,0,True,False,False,ja,[https://t.co/OpUyzGpJxE],https://twitter.com/922613564/status/156136752...,RT @USER_NAME: 韓国紙「日本経済を抜くと言ったな？あれは嘘だ、それどころかフィ...
3,2022-08-22 00:00:01+09:00,1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...,1507665461353533440,Twitter Web App,ごうき | ビジネス構築のプロ,goki_consultant,0,1,4,15,False,False,False,ja,[],https://twitter.com/1507665461353533449/status...,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...
4,2022-08-22 00:00:01+09:00,1561367524259663872,RT @HANADA_asuka: 【「旭日旗＝戦争犯罪」という嘘】\n韓国のプロパガンダの...,1045097019855921152,Twitter for iPhone,岡内 隆之,2OqmyL0zhgWN8lf,0,0,514,0,True,False,False,ja,[],https://twitter.com/1045097019855921153/status...,RT @USER_NAME: 【「旭日旗＝戦争犯罪」という嘘】\n韓国のプロパガンダの元にな...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,2022-08-28 23:59:57+09:00,1563904224895053824,RT @abetokyo: あの、、僕は一貫して「施設に入場する際は」としか言ってないんです...,1439932155497439232,Twitter for Android,どーん,4omyre0JWu2xGEH,0,0,286,0,True,False,False,ja,[],https://twitter.com/1439932155497439235/status...,RT @USER_NAME: あの、、僕は一貫して「施設に入場する際は」としか言ってないんで...
1197,2022-08-28 23:59:58+09:00,1563904227914948608,@Rin_T_T_2 ん？もう寝るよ(嘘),1563865048698978304,Twitter for iPhone,玲王大好きman,Reidaisukima,0,1,0,1,False,False,True,ja,[],https://twitter.com/1563865048698978306/status...,@USER_NAME ん？もう寝るよ(嘘)
1198,2022-08-28 23:59:58+09:00,1563904228266893312,RT @dermar_gg: ねぇ和くん…もう一度だけ彼女に\nしてくれないかな…💦（嘘泣）...,738166718539784192,Twitter Web App,CreativeGie,_CreativeGie,0,0,249,0,True,False,False,ja,[https://t.co/X7dFuzqsQt],https://twitter.com/738166718539784192/status/...,RT @USER_NAME: ねぇ和くん…もう一度だけ彼女に\nしてくれないかな…💦（嘘泣）...
1199,2022-08-28 23:59:58+09:00,1563904228619595776,RT @garirou: 大きな組織が嘘をつく筈がないと考える人は、（大きな組織という名の）...,1423872035063697408,Twitter for Android,トカレフ,T3485KV1ASU122,0,0,91,0,True,False,False,ja,[],https://twitter.com/1423872035063697411/status...,RT @USER_NAME: 大きな組織が嘘をつく筈がないと考える人は、（大きな組織という名...


# データクリーニング

In [5]:
# まずはRTについて考える
# RTはオリジナルツイートの複製なので、複製された情報が不要な場合は取り除いたほうが計算量を削減できる
original_tweets = df.query("is_rt == False")
display(original_tweets)

,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
3,2022-08-22 00:00:01+09:00,1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...,1507665461353533440,Twitter Web App,ごうき | ビジネス構築のプロ,goki_consultant,0,1,4,15,False,False,False,ja,[],https://twitter.com/1507665461353533449/status...,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...
5,2022-08-22 00:00:01+09:00,1561367524348162048,@love_honey_93 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨,1526555598090915840,Twitter for Android,🐰よぅ◠‿◠ちゃん🦌,mame951230,0,1,0,1,False,False,True,ja,[],https://twitter.com/1526555598090915843/status...,@USER_NAME 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨
6,2022-08-22 00:00:02+09:00,1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘...,1152060130768244736,Twitter for iPhone,され妻ちゃン,zqvnr,0,0,0,2,False,False,False,ja,[],https://twitter.com/1152060130768244738/status...,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘...
11,2022-08-22 00:00:03+09:00,1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PC...,3008013404,Twitter Web App,よん,yo_nnn,0,1,0,2,False,False,True,ja,[],https://twitter.com/3008013404/status/15613675...,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PC...
12,2022-08-22 00:00:03+09:00,1561367533412044800,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は...,1451434811738914816,Twitter for Android,らーゆ ／ 🎃参列済,walunt_1347,0,5,1,8,False,False,False,ja,[],https://twitter.com/1451434811738914821/status...,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,2022-08-28 23:59:53+09:00,1563904207815512064,@na_o70725 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時...,172804761,Twitter for iPhone,たけちゃん,Via_Landeta_CA,0,2,7,68,False,False,True,ja,[],https://twitter.com/172804761/status/156390420...,@USER_NAME 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時...
1191,2022-08-28 23:59:54+09:00,1563904209459613696,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上...,1399513807,twittbot.net,来太のbot,BotRaita,0,0,0,0,False,False,False,ja,[],https://twitter.com/1399513807/status/15639042...,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上...
1192,2022-08-28 23:59:54+09:00,1563904211074822144,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) https://t.co/C...,854988257926365184,Nintendo Switch Share,シンプルイズザベスト,64DSWiiGBA,0,0,0,2,False,False,False,ja,"[https://t.co/Cf8yyUO59K, https://t.co/Cf8yyUO...",https://twitter.com/854988257926365184/status/...,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) <URL>
1194,2022-08-28 23:59:57+09:00,1563904221845409792,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失...,84337504,Twitter Web App,鳥酋長,bird_chief,0,0,0,0,False,False,False,ja,[],https://twitter.com/84337504/status/1563904221...,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失...


In [6]:
# 割合を計算する関数を定義
def _rate(df_a, df_b):
    r = round(len(df_a) / len(df_b), 2) * 100
    print(f"元データに対して{r}%")
    display(df_a)

In [7]:
# RTを除くと元のデータに対して半分以下まで減る
# 単純に考えれば実行時間やメモリの消費量は半分になるので、本当に自分の用途にRTは必要か？は早い段階で考えたほうが良いことがわかる
_rate(original_tweets, df)

元データに対して44.0%


,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
3,2022-08-22 00:00:01+09:00,1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...,1507665461353533440,Twitter Web App,ごうき | ビジネス構築のプロ,goki_consultant,0,1,4,15,False,False,False,ja,[],https://twitter.com/1507665461353533449/status...,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸...
5,2022-08-22 00:00:01+09:00,1561367524348162048,@love_honey_93 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨,1526555598090915840,Twitter for Android,🐰よぅ◠‿◠ちゃん🦌,mame951230,0,1,0,1,False,False,True,ja,[],https://twitter.com/1526555598090915843/status...,@USER_NAME 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨
6,2022-08-22 00:00:02+09:00,1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘...,1152060130768244736,Twitter for iPhone,され妻ちゃン,zqvnr,0,0,0,2,False,False,False,ja,[],https://twitter.com/1152060130768244738/status...,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘...
11,2022-08-22 00:00:03+09:00,1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PC...,3008013404,Twitter Web App,よん,yo_nnn,0,1,0,2,False,False,True,ja,[],https://twitter.com/3008013404/status/15613675...,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PC...
12,2022-08-22 00:00:03+09:00,1561367533412044800,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は...,1451434811738914816,Twitter for Android,らーゆ ／ 🎃参列済,walunt_1347,0,5,1,8,False,False,False,ja,[],https://twitter.com/1451434811738914821/status...,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,2022-08-28 23:59:53+09:00,1563904207815512064,@na_o70725 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時...,172804761,Twitter for iPhone,たけちゃん,Via_Landeta_CA,0,2,7,68,False,False,True,ja,[],https://twitter.com/172804761/status/156390420...,@USER_NAME 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時...
1191,2022-08-28 23:59:54+09:00,1563904209459613696,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上...,1399513807,twittbot.net,来太のbot,BotRaita,0,0,0,0,False,False,False,ja,[],https://twitter.com/1399513807/status/15639042...,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上...
1192,2022-08-28 23:59:54+09:00,1563904211074822144,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) https://t.co/C...,854988257926365184,Nintendo Switch Share,シンプルイズザベスト,64DSWiiGBA,0,0,0,2,False,False,False,ja,"[https://t.co/Cf8yyUO59K, https://t.co/Cf8yyUO...",https://twitter.com/854988257926365184/status/...,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) <URL>
1194,2022-08-28 23:59:57+09:00,1563904221845409792,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失...,84337504,Twitter Web App,鳥酋長,bird_chief,0,0,0,0,False,False,False,ja,[],https://twitter.com/84337504/status/1563904221...,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失...


In [8]:
# 次はBOTについて考える

# Twitter上で@から始まるユーザの識別子scree_nameで重複削除
dedupe_by_screen_name = original_tweets.drop_duplicates("screen_name")
# screen_nameが_botで終わるアカウントはBOTと仮定
bot_tweet = dedupe_by_screen_name[
    dedupe_by_screen_name["screen_name"].str.endswith("_bot")
]

# デマやウソなどの投稿を含むツイートはもしかするとBOTが多いかもしれないので見てみる
# 参考までに、他のチュートリアルのデータでは0.0001以下のものもある
bot_rate = len(bot_tweet) / len(dedupe_by_screen_name)
print("ユニークユーザ数:", len(dedupe_by_screen_name))
print("botが占める割合:", round(bot_rate, 5))

ユニークユーザ数: 296140
botが占める割合: 0.02104


In [9]:
# 3000アカウント近くはBOTかもしれない。さらに、screen_nameが_botで終わるとは限らない、どうやってあぶり出すか？
# ユーザがツイートを投稿する際にはそのTwitterClientが記録されているのでそれを使う
# まずはbotとそれ以外でTwitterClientの頻度表を作る

compare_df = pandas.DataFrame()
n = 10
compare_df["all"] = dedupe_by_screen_name["source"].value_counts().head(n).index
compare_df["bot"] = bot_tweet["source"].value_counts().head(n).index
display(compare_df)

,all,bot
0,Twitter for iPhone,twittbot.net
1,Twitter for Android,Botbird tweets
2,Twitter Web App,Twitter for iPhone
3,twittbot.net,Twitter for Android
4,Twitter for iPad,Twitter Web App
5,Botbird tweets,Twittbot Web
6,TweetDeck,Twitter
7,Peing,邪無夢bot
8,feather for iOS,TwiBotMaker
9,SocialDog for Twitter,IFTTT


In [10]:
# botに顕著に多いものを見ていくと　twittbot.net　というBOTのホスティングサービスがあることがわかる
# 「嘘」というクエリが一般的すぎるため、キャラクターBOTに引っかかる影響も見える
# 参考：https://twittbot.net/
dedupe_by_screen_name.query("source == 'twittbot.net'").head(5).loc[:, ["name", "text"]]

,name,text
25,ななこP,す、好きじゃなくも無いけど？ う、う嘘、嘘 ……嫌いじゃないよ
39,てとちあ・ちあてとコピペbot,千秋：たまにな、『ああ誰でも良いから思いっきり甘えたい』って思うときがあるんだ。それで、そう...
85,バトロワコピペbot,【クイズ】 \n桐山「嘘をついて地獄に落ちると閻魔大王に何をされる？」笹川「カンチョー」黒...
87,佐方直之(bot),「今度はね、見えているものを、嘘偽りなく描こうと思うんだ。前までは失われたものを描こうとした...
92,蒼崎 暗夜,0時。僕はそろそろ寝る、けど…一緒に寝る？…なんて、う、嘘だよ！おやすみ…！///


In [11]:
# これも同じく
# また、人のようなアカウントも実はBOTサービスから投稿されていることがわかる

pandas.set_option("display.max_colwidth", None)

dedupe_by_screen_name.query("source == 'Botbird tweets'").head(5).loc[
    :, ["name", "text", "link"]
]

,name,text,link
71,neS4YRtLBhnc4Zl,今日は新型コロナウィルスのデマについて、株式会社メディアコラボ代表の古田大輔さんから武田砂鉄さんがお話を伺います\n\n武田：具体的に特効薬のデマはどんな感じですか？\n古田：面白いことに国を跨いで同じデマが流れているんですが「大麻で治る！」とか流れてますね。 https://t.co/Nyf9jxV39u https://t.co/FPS4tHDHzs,https://twitter.com/1208537996682612736/status/1561367624109494279
190,re inbow,https://t.co/kGuwbxc0d8 T.A.T.U - All the things she said ( live 2013 Russia) ＃ニセモノ #やらせ #フェイク #standwithrussia #ティプシー #1d #inizon,https://twitter.com/1548657728746569728/status/1561367846843809794
510,超合金ZZ,日本人は大勢のサイレントマジョリティとごく一部のネトウヨ、ブサヨ、放射脳、バカッターなどといった声がでかいだけで中身の無いノイジーマイノリティで構成されている。\n困ったことに日本人は声が大きければ嘘でも受け入れられるという難点を抱えている。,https://twitter.com/40390387/status/1561368327368462337
570,ゆっくり彰晃bot,@karetabero_kai 嘘も100回言えば本当になる,https://twitter.com/1558693785844199424/status/1561368431299072004
1291,らっきー,自分が思うより恋をしていたあなたに\nあれから思うように息ができない\nあんなに側にいたのにまるで嘘みたい\nとても忘れられないそれだけが確か,https://twitter.com/1334019265985343488/status/1561369571659063296


In [12]:
# こういったサービスのリストが作れると理想ではあるが、再現がなくそれだけで大きな作業になってしまう
# 割り切って公式のTwitterClientのみ使うホワイトリスト方式にする方法もある

official_clients = [
    "Twitter for iPhone",
    "Twitter for Android",
    "Twitter Web App",
    "Twitter for iPad",
    "Twitter Web App",
    "TweetDeck",
]
filtered_by_source = original_tweets.query("source in @official_clients")

# 参考までにデータによっては88%程度になったりするので量としては問題がなさそうに見える
# ただし、削るかどうかは「検証したい仮説への影響」と「省ける作業コスト」を提示した上で議論して決めること
_rate(filtered_by_source, original_tweets)

元データに対して75.0%


,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
3,2022-08-22 00:00:01+09:00,1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸嘘ついて仕事を取らない❹相手を本気で勝たせる❺法人化すると信頼される❻不安にさせない報連相…この6つを意識してみて。継続案件は貰えるし、案件は増えるしでウハウハになるから！#おは戦40822an,1507665461353533440,Twitter Web App,ごうき | ビジネス構築のプロ,goki_consultant,0,1,4,15,False,False,False,ja,[],https://twitter.com/1507665461353533449/status/1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸嘘ついて仕事を取らない❹相手を本気で勝たせる❺法人化すると信頼される❻不安にさせない報連相…この6つを意識してみて。継続案件は貰えるし、案件は増えるしでウハウハになるから！#おは戦40822an
5,2022-08-22 00:00:01+09:00,1561367524348162048,@love_honey_93 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨,1526555598090915840,Twitter for Android,🐰よぅ◠‿◠ちゃん🦌,mame951230,0,1,0,1,False,False,True,ja,[],https://twitter.com/1526555598090915843/status/1561367524348162048,@USER_NAME 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨
6,2022-08-22 00:00:02+09:00,1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘に見えて聞こえる,1152060130768244736,Twitter for iPhone,され妻ちゃン,zqvnr,0,0,0,2,False,False,False,ja,[],https://twitter.com/1152060130768244738/status/1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘に見えて聞こえる
11,2022-08-22 00:00:03+09:00,1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PCは絶対に七不思議に行け……絶対だ……絶対に行け……いいか……よんとの約束だぞ……行かないと後悔するまである……\n次が本当の本当に！！最終日ですのよ！！七日目にして解決ですわ！！！やりますわよ！！,3008013404,Twitter Web App,よん,yo_nnn,0,1,0,2,False,False,True,ja,[],https://twitter.com/3008013404/status/1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PCは絶対に七不思議に行け……絶対だ……絶対に行け……いいか……よんとの約束だぞ……行かないと後悔するまである……\n次が本当の本当に！！最終日ですのよ！！七日目にして解決ですわ！！！やりますわよ！！
12,2022-08-22 00:00:03+09:00,1561367533412044800,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は)\nなんならチーズケーキくれても…\n\n嘘ですすいません調子乗りました（\nこれからもよろしくぅ！！！,1451434811738914816,Twitter for Android,らーゆ ／ 🎃参列済,walunt_1347,0,5,1,8,False,False,False,ja,[],https://twitter.com/1451434811738914821/status/1561367533412044812,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は)\nなんならチーズケーキくれても…\n\n嘘ですすいません調子乗りました（\nこれからもよろしくぅ！！！
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,2022-08-28 23:59:49+09:00,1563904191038648320,確定ファンサ何回も何回もほんとにありがとうございました（ ; ; ）\n\n今回はもう肉眼でジェルくんの笑顔見られたらそれでいいかなって思ってたんだけど自分でも嘘なんじゃないかと思うくらい、何度も見つけてもらえてうれしかった\n\n#すとぷり西武ドームDAY2\n#ジェルくん https://t.co/aS3wrkgvjI,1192000885771030528,Twitter for iPhone,ここ。,koko__1016,0,0,0,9,False,False,False,ja,[https://t.co/aS3wrkgvjI],https://twitter.com/1192000885771030529/status/1563904191038648320,確定ファンサ何回も何回もほんとにありがとうございました（ ; ; ）\n\n今回はもう肉眼でジェルくんの笑顔見られたらそれでいいかなって思ってたんだけど自分でも嘘なんじゃないかと思うくらい、何度も見つけてもらえてうれしかった\n\n#すとぷり西武ドームDAY2\n#ジェルくん <URL>
1184,2022-08-28 23:59:50+09:00,1563904194184364032,@MEGU5979 @neu_hanp まあ、情勢報告も大概嘘なんですけどねwww,2617495038,Twitter for iPhone,ワタセユウヤ,yuyawatase,0,1,3,14,False,False,True,ja,[],https://twitter.com/2617495038/status/1563904194184364032,@USER_NAME @USER_NAME まあ、情勢報告も大概嘘なんですけどねwww
1189,2022-08-28 23:59:53+09:00,1563904207815512064,@na_o70725 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時意識が無くなりましたが、1週間も経てば嘘の様にケロッと治りました。私の主治医も原因はワクだなと。人によっては何か別の悪さをするモノを私達は接種しているのかも知れませんね。,172804761,Twitter for iPhone,たけちゃん,Via_Landeta_CA,0,2,7,68,False,False,True,ja,[],https://twitter.com/172804761/status/1563904207815512065,@USER_NAME 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時意識が無くなりましたが、1週間も経てば嘘の様にケロッと治りました。私の主治医も原因はワクだなと。人によっては何か別の悪さをするモノを私達は接種しているのかも知れませんね。
1194,2022-08-28 23:59:57+09:00,1563904221845409792,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失礼だとまで思ってる。\n…ウソですほんとは超やりたいです。,84337504,Twitter Web App,鳥酋長,bird_chief,0,0,0,0,False,False,False,ja,[],https://twitter.com/84337504/status/1563904221845409794,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失礼だとまで思ってる。\n…ウソですほんとは超やりたいです。


In [13]:
# クエリが一般的すぎる場合は部分一致で意図しないツイートまで拾ってしまう
# しかし、除外するクエリを考えるのは大変なので、ある程度機械的に絞り込む方法を考える

# 部分一致の多くは名詞にマッチしてしまうことを利用してwikipediaのページから名詞のリストを作る
# 作成に20分程度かかるので事前に作っておいたものをダウンロードして使う
wiki_pageview = pandas.read_csv(
    "https://drive.google.com/u/0/uc?id=1sxMZtnu0fHMJKuKzG9Fni_FsK-qzRzHg&export=download",
    sep="\t",
    na_filter=False,
)
# entry:ページ名
# count:ある1日の間で閲覧された数
# length:ページ名の長さ
wiki_pageview.sort_values("count", ascending=False)

,entry,count,length
111827,メインページ,539736,6
221796,特別:検索,112181,5
253591,辻元舞,67852,3
213508,池田満寿夫,57981,5
14,-,56439,1
...,...,...,...
121000,ロチェスターエレクトロニクス,1,14
976726,キモット,1,4
468028,ポーランド継承戦争_(1587年-1588年),1,23
976724,キム・ミンソク_(フィギュアスケート選手),1,21


In [14]:
# 文字列を与えると除外するべき文字列の候補を提案する関数
def not_query_suggest(df, keyword, n):
    filtered = df[df["entry"].str.contains(keyword)]
    top_n = pandas.DataFrame()
    # 文字列が短い場合は部分一致しやすいという仮説
    top_n["sort_by_length"] = [e for e in filtered.sort_values("length")["entry"]]
    # 閲覧数が多い場合はそれだけツイート上にも出現しやすいという仮説
    top_n["sort_by_count"] = [
        e for e in filtered.sort_values("count", ascending=False)["entry"]
    ]
    print(keyword)
    display(top_n.head(n))


# 嘘　を例に除外したほうが良い文字列を出力
not_query_suggest(wiki_pageview, "嘘", 10)

嘘


,sort_by_length,sort_by_count
0,嘘,カノジョは嘘を愛しすぎてる
1,嘘松,四月は君の嘘
2,嘘字,嘘喰い
3,嘘く,恋と嘘
4,嘘八百,愛しい嘘_優しい闇
5,嘘つき,嘘松
6,藤城嘘,嘘の証明_犯罪心理分析官・梶原圭子
7,嘘の木,嘘をつく子供
8,嘘食い,霊能力者_小田霧響子の嘘
9,恋と嘘,嘘から始まる恋


In [15]:
# 自動でやってしまうと、「嘘八百」や「嘘つき」などが削られてしまうことがわかる
# ここは手動で選んで、作品名と思われる文字列をえられたので試してみる
ng_words = ["カノジョは嘘を愛しすぎてる", "四月は君の嘘", "嘘喰い", "恋と嘘", "嘘から始まる恋"]


def should_exclude(text):
    for w in ng_words:
        if w in text:
            return True
    return False


# 除外を適応した場合、なくなるツイートの一覧
# Twitterの性質上、サブカルチャーに属するものはかなり拾ってしまっていることがわかる
original_tweets[original_tweets["text"].apply(should_exclude)]

,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
234,2022-08-22 00:01:37+09:00,1561367929538560000,届くかな 届くといいな 有馬公生君 最終話「四月は君の嘘」,1443889680,twittbot.net,四月は君の嘘bot,conaufk,0,0,0,0,False,False,False,ja,[],https://twitter.com/1443889680/status/1561367929538560000,届くかな 届くといいな 有馬公生君 最終話「四月は君の嘘」
439,2022-08-22 00:02:49+09:00,1561368229833953280,買え！そこのキングダムと嘘喰いの間に挟んで！さぁ！,1572771782,twittbot.net,横田たかしbot,wxy_takashi_bot,0,0,0,0,False,False,False,ja,[],https://twitter.com/1572771782/status/1561368229833953280,買え！そこのキングダムと嘘喰いの間に挟んで！さぁ！
50,2022-08-22 00:10:24+09:00,1561370138934255616,嘘喰い(実写版)の話からの\n私「似たテイストの原作で言うとライアーゲームは面白いと思いました。戸田恵子が可愛いですし！」\nお綺麗なのは事実ですが出演してませんでした…,1068962718336483328,Twitter for Android,snbijr,snbijr,0,0,0,4,False,False,False,ja,[],https://twitter.com/1068962718336483328/status/1561370138934255617,嘘喰い(実写版)の話からの\n私「似たテイストの原作で言うとライアーゲームは面白いと思いました。戸田恵子が可愛いですし！」\nお綺麗なのは事実ですが出演してませんでした…
460,2022-08-22 00:13:11+09:00,1561370838200647680,ミュージカル四月は君の嘘再演してクレメンス,1309448118589554688,Twitter for iPhone,と星さ2/1730,4_s6v,0,1,0,1,False,False,False,ja,[],https://twitter.com/1309448118589554688/status/1561370838200647680,ミュージカル四月は君の嘘再演してクレメンス
823,2022-08-22 00:15:36+09:00,1561371446848651264,@Kamui_DX 四月は君の嘘、見ましたか？,129851611,TweetDeck,𝑩𝒆𝒆𝒓 🍻,Win_DDR,0,1,0,1,False,False,True,ja,[],https://twitter.com/129851611/status/1561371446848651266,@USER_NAME 四月は君の嘘、見ましたか？
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,2022-08-28 23:37:35+09:00,1563898596902436864,ぐでAliceさんを構成する5つのマンガは\nマーブルビターチョコレート\n恋と嘘\n地縛少年 花子くん\n一生好きってゆったじゃん\nクズの本懐\nでした！ #私を構成する5つのマンガ @alu_inc より https://t.co/X88C4d5yKC\n\n読も…………?,546019755,Twitter for iPhone,ぐでAlice@色塗りして👏,Alice071227,0,1,0,0,False,False,False,ja,[https://t.co/X88C4d5yKC],https://twitter.com/546019755/status/1563898596902436864,ぐでAliceさんを構成する5つのマンガは\nマーブルビターチョコレート\n恋と嘘\n地縛少年 花子くん\n一生好きってゆったじゃん\nクズの本懐\nでした！ #私を構成する5つのマンガ @USER_NAME より <URL>\n\n読も…………?
518,2022-08-28 23:44:19+09:00,1563900290277216256,四月は君の嘘、ガチ泣けるので全人類に見て欲しいと思うほんと。,1334505297168261120,Twitter for iPhone,まよい🐌❤️,SC_mayo,0,3,0,11,False,False,False,ja,[],https://twitter.com/1334505297168261123/status/1563900290277216257,四月は君の嘘、ガチ泣けるので全人類に見て欲しいと思うほんと。
1157,2022-08-28 23:49:57+09:00,1563901707133693952,@nukegeverymuch 草 俺の場合推しの子とかぐや様と恋と嘘被ってるから3万6千くらいですね(血反吐),1530103187897655296,Twitter for Android,りんの.sub,rinno0525,0,1,0,0,False,False,True,ja,[],https://twitter.com/1530103187897655296/status/1563901707133693952,@USER_NAME 草 俺の場合推しの子とかぐや様と恋と嘘被ってるから3万6千くらいですね(血反吐)
788,2022-08-28 23:56:43+09:00,1563903408695783424,よしもうすぐ嘘喰い読めるうおおおお,911864403271704576,Twitter for Android,なめこんぶ,RedstonerNK,0,0,0,0,False,False,False,ja,[],https://twitter.com/911864403271704576/status/1563903408695783424,よしもうすぐ嘘喰い読めるうおおおお


In [16]:
# 他の例も見る
not_query_suggest(wiki_pageview, "デマ", 10)
not_query_suggest(wiki_pageview, "ウソ", 10)
not_query_suggest(wiki_pageview, "フェイク", 10)
not_query_suggest(wiki_pageview, "ガセ", 10)

デマ


,sort_by_length,sort_by_count
0,デマ,ソフト・オン・デマンド
1,コデマリ,ビデオ・オン・デマンド
2,デマント,NHKオンデマンド
3,アペデマク,デマゴーグ
4,アデマール,ASアデマ_149-0_SOレミルヌ
5,デマラトス,フジテレビオンデマンド
6,リンデマン,オンデマンド
7,オオデマリ,デマー・デローザン
8,ヤブデマリ,ウィーデマン・フランツの法則
9,デマゴギー,オーデマ・ピゲ


ウソ


,sort_by_length,sort_by_count
0,ウソ,ウソップ
1,ウソ技,騎士竜戦隊リュウソウジャー
2,ウソク,カワウソ
3,ウソツキ,ニホンカワウソ
4,ウソハチ,チョン・ウソン
5,ウソコイ,キム・ウソク
6,ウソップ,キュウソネコカミ
7,ラウソン,コツメカワウソ
8,ロウソク,トウキョウソナタ
9,カワウソ,ギンリョウソウ


フェイク


,sort_by_length,sort_by_count
0,フェイク,ギャラリーフェイク
1,フェイク着信,フェイク_(映画)
2,フェイクロア,ディープフェイク
3,フェイクライカ,ダブルフェイク_アンダー・ザ・ガンダム
4,フェイクファー,フェイクファー_(アルバム)
5,ディープフェイク,フェイク_(Mr.Childrenの曲)
6,フェイク・ブック,フェイク
7,スタジオフェイク,フェイクニュース_あるいはどこか遠くの戦争の話
8,フェイクニュース,フェイクロア
9,ギャラリーフェイク,フェイク_シティ_ある男のルール


ガセ


,sort_by_length,sort_by_count
0,ナガセ,ナガセ
1,ガセネタ,ホセ・オルテガ・イ・ガセット
2,ガセリ菌,サルオガセモドキ
3,ガチガセ,サルオガセ
4,サルオガセ,ガセリ菌
5,ナガセケイ,ガセリ菌SP株
6,カンガセイロ,ガセネタ_(バンド)
7,ガセリ菌SP株,ナガセケイ
8,アブダガセス1世,カンガセイロ
9,ヴォロガセス2世,ガセネタ


In [17]:
# 除外するリストを拡張して最終的に以下のリストを作る
ng_words.extend(["デマンド", "デマント", "アペデマク", "アデマール"])
ng_words.extend(["ウソク", "ウソップ", "リュウソウジャー", "カワウソ", "ロウソク"])
ng_words.extend(["フェイクファー", "ギャラリーフェイク", "フェイクロア"])
ng_words.extend(["ナガセ", "ガセリ菌"])
ng_words

['カノジョは嘘を愛しすぎてる',
 '四月は君の嘘',
 '嘘喰い',
 '恋と嘘',
 '嘘から始まる恋',
 'デマンド',
 'デマント',
 'アペデマク',
 'アデマール',
 'ウソク',
 'ウソップ',
 'リュウソウジャー',
 'カワウソ',
 'ロウソク',
 'フェイクファー',
 'ギャラリーフェイク',
 'フェイクロア',
 'ナガセ',
 'ガセリ菌']

In [18]:
# 適用すると削ることのできるツイートのリストを見てみる
original_tweets[original_tweets["text"].apply(should_exclude)]

,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
234,2022-08-22 00:01:37+09:00,1561367929538560000,届くかな 届くといいな 有馬公生君 最終話「四月は君の嘘」,1443889680,twittbot.net,四月は君の嘘bot,conaufk,0,0,0,0,False,False,False,ja,[],https://twitter.com/1443889680/status/1561367929538560000,届くかな 届くといいな 有馬公生君 最終話「四月は君の嘘」
439,2022-08-22 00:02:49+09:00,1561368229833953280,買え！そこのキングダムと嘘喰いの間に挟んで！さぁ！,1572771782,twittbot.net,横田たかしbot,wxy_takashi_bot,0,0,0,0,False,False,False,ja,[],https://twitter.com/1572771782/status/1561368229833953280,買え！そこのキングダムと嘘喰いの間に挟んで！さぁ！
50,2022-08-22 00:10:24+09:00,1561370138934255616,嘘喰い(実写版)の話からの\n私「似たテイストの原作で言うとライアーゲームは面白いと思いました。戸田恵子が可愛いですし！」\nお綺麗なのは事実ですが出演してませんでした…,1068962718336483328,Twitter for Android,snbijr,snbijr,0,0,0,4,False,False,False,ja,[],https://twitter.com/1068962718336483328/status/1561370138934255617,嘘喰い(実写版)の話からの\n私「似たテイストの原作で言うとライアーゲームは面白いと思いました。戸田恵子が可愛いですし！」\nお綺麗なのは事実ですが出演してませんでした…
460,2022-08-22 00:13:11+09:00,1561370838200647680,ミュージカル四月は君の嘘再演してクレメンス,1309448118589554688,Twitter for iPhone,と星さ2/1730,4_s6v,0,1,0,1,False,False,False,ja,[],https://twitter.com/1309448118589554688/status/1561370838200647680,ミュージカル四月は君の嘘再演してクレメンス
565,2022-08-22 00:13:49+09:00,1561370998834106368,@haru2292 byロロノア・フロイ\n嘘嘘🤥ウソップ\n一緒に二人三脚でいこ笑足に縄つけて,1027420453646659584,Twitter for iPhone,Froi @ナツヨメ,froIDA18,0,1,0,2,False,False,True,ja,[],https://twitter.com/1027420453646659584/status/1561370998834106368,@USER_NAME byロロノア・フロイ\n嘘嘘🤥ウソップ\n一緒に二人三脚でいこ笑足に縄つけて
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,2022-08-28 23:37:35+09:00,1563898596902436864,ぐでAliceさんを構成する5つのマンガは\nマーブルビターチョコレート\n恋と嘘\n地縛少年 花子くん\n一生好きってゆったじゃん\nクズの本懐\nでした！ #私を構成する5つのマンガ @alu_inc より https://t.co/X88C4d5yKC\n\n読も…………?,546019755,Twitter for iPhone,ぐでAlice@色塗りして👏,Alice071227,0,1,0,0,False,False,False,ja,[https://t.co/X88C4d5yKC],https://twitter.com/546019755/status/1563898596902436864,ぐでAliceさんを構成する5つのマンガは\nマーブルビターチョコレート\n恋と嘘\n地縛少年 花子くん\n一生好きってゆったじゃん\nクズの本懐\nでした！ #私を構成する5つのマンガ @USER_NAME より <URL>\n\n読も…………?
518,2022-08-28 23:44:19+09:00,1563900290277216256,四月は君の嘘、ガチ泣けるので全人類に見て欲しいと思うほんと。,1334505297168261120,Twitter for iPhone,まよい🐌❤️,SC_mayo,0,3,0,11,False,False,False,ja,[],https://twitter.com/1334505297168261123/status/1563900290277216257,四月は君の嘘、ガチ泣けるので全人類に見て欲しいと思うほんと。
1157,2022-08-28 23:49:57+09:00,1563901707133693952,@nukegeverymuch 草 俺の場合推しの子とかぐや様と恋と嘘被ってるから3万6千くらいですね(血反吐),1530103187897655296,Twitter for Android,りんの.sub,rinno0525,0,1,0,0,False,False,True,ja,[],https://twitter.com/1530103187897655296/status/1563901707133693952,@USER_NAME 草 俺の場合推しの子とかぐや様と恋と嘘被ってるから3万6千くらいですね(血反吐)
788,2022-08-28 23:56:43+09:00,1563903408695783424,よしもうすぐ嘘喰い読めるうおおおお,911864403271704576,Twitter for Android,なめこんぶ,RedstonerNK,0,0,0,0,False,False,False,ja,[],https://twitter.com/911864403271704576/status/1563903408695783424,よしもうすぐ嘘喰い読めるうおおおお


In [19]:
# 数字上では影響は小さいが、共起マップを作るときに変なクラスターができるのを緩和できる
original_tweets[~original_tweets["text"].apply(should_exclude)]

,created_at,id_str,text,author_id,source,name,screen_name,quote_count,reply_count,retweet_count,like_count,is_rt,is_qt,is_reply,lang,urls,link,masked
3,2022-08-22 00:00:01+09:00,1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸嘘ついて仕事を取らない❹相手を本気で勝たせる❺法人化すると信頼される❻不安にさせない報連相…この6つを意識してみて。継続案件は貰えるし、案件は増えるしでウハウハになるから！#おは戦40822an,1507665461353533440,Twitter Web App,ごうき | ビジネス構築のプロ,goki_consultant,0,1,4,15,False,False,False,ja,[],https://twitter.com/1507665461353533449/status/1561367526105182208,フリーランスに伝えたい。成果を出す秘訣は…❶実績は盛らない❷目の前のお客さんにフルコミット❸嘘ついて仕事を取らない❹相手を本気で勝たせる❺法人化すると信頼される❻不安にさせない報連相…この6つを意識してみて。継続案件は貰えるし、案件は増えるしでウハウハになるから！#おは戦40822an
5,2022-08-22 00:00:01+09:00,1561367524348162048,@love_honey_93 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨,1526555598090915840,Twitter for Android,🐰よぅ◠‿◠ちゃん🦌,mame951230,0,1,0,1,False,False,True,ja,[],https://twitter.com/1526555598090915843/status/1561367524348162048,@USER_NAME 嘘ついたww\n前日ではないと思う！\nきっと買えるよ✨
6,2022-08-22 00:00:02+09:00,1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘に見えて聞こえる,1152060130768244736,Twitter for iPhone,され妻ちゃン,zqvnr,0,0,0,2,False,False,False,ja,[],https://twitter.com/1152060130768244738/status/1561367529200553984,信じてた人に裏切られたらこんなに信用できやんくなるもんなんやなと改めて感じた\nぜーんぶが嘘に見えて聞こえる
11,2022-08-22 00:00:03+09:00,1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PCは絶対に七不思議に行け……絶対だ……絶対に行け……いいか……よんとの約束だぞ……行かないと後悔するまである……\n次が本当の本当に！！最終日ですのよ！！七日目にして解決ですわ！！！やりますわよ！！,3008013404,Twitter Web App,よん,yo_nnn,0,1,0,2,False,False,True,ja,[],https://twitter.com/3008013404/status/1561367531696582656,≪６日目≫\n次が最終日だといいましたわね？あれは嘘ですわ(\nもう……もう……天落通過PCは絶対に七不思議に行け……絶対だ……絶対に行け……いいか……よんとの約束だぞ……行かないと後悔するまである……\n次が本当の本当に！！最終日ですのよ！！七日目にして解決ですわ！！！やりますわよ！！
12,2022-08-22 00:00:03+09:00,1561367533412044800,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は)\nなんならチーズケーキくれても…\n\n嘘ですすいません調子乗りました（\nこれからもよろしくぅ！！！,1451434811738914816,Twitter for Android,らーゆ ／ 🎃参列済,walunt_1347,0,5,1,8,False,False,False,ja,[],https://twitter.com/1451434811738914821/status/1561367533412044812,っしゃ、誕生日じゃ！！！！！！\nまたひとつ腐女子歴が伸びたのぉ…。\n\n皆の衆、祝え(は)\nなんならチーズケーキくれても…\n\n嘘ですすいません調子乗りました（\nこれからもよろしくぅ！！！
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,2022-08-28 23:59:53+09:00,1563904207815512064,@na_o70725 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時意識が無くなりましたが、1週間も経てば嘘の様にケロッと治りました。私の主治医も原因はワクだなと。人によっては何か別の悪さをするモノを私達は接種しているのかも知れませんね。,172804761,Twitter for iPhone,たけちゃん,Via_Landeta_CA,0,2,7,68,False,False,True,ja,[],https://twitter.com/172804761/status/1563904207815512065,@USER_NAME 心中お察しします。私も接種後、何の前触れも無く喘息になり🚑と入院。一時意識が無くなりましたが、1週間も経てば嘘の様にケロッと治りました。私の主治医も原因はワクだなと。人によっては何か別の悪さをするモノを私達は接種しているのかも知れませんね。
1191,2022-08-28 23:59:54+09:00,1563904209459613696,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上に広がって。シロはいつしかクロばかり。少しのホントで芽生えるシロは、クロの上ではクロとなる。クロの上では全部クロ。\nクロばかりになった私の心。今はクロがクロかも分からない。,1399513807,twittbot.net,来太のbot,BotRaita,0,0,0,0,False,False,False,ja,[],https://twitter.com/1399513807/status/1563904209459613698,シロばかりだった私の心。そこに芽生えた小さな小さなクロの点。一つのウソがクロとなり、シロの上に広がって。シロはいつしかクロばかり。少しのホントで芽生えるシロは、クロの上ではクロとなる。クロの上では全部クロ。\nクロばかりになった私の心。今はクロがクロかも分からない。
1192,2022-08-28 23:59:54+09:00,1563904211074822144,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) https://t.co/Cf8yyUO59K,854988257926365184,Nintendo Switch Share,シンプルイズザベスト,64DSWiiGBA,0,0,0,2,False,False,False,ja,"[https://t.co/Cf8yyUO59K, https://t.co/Cf8yyUO59K]",https://twitter.com/854988257926365184/status/1563904211074822145,前夜祭、結果発表！\n何だか異次元な結果が出ましたね！(大嘘) <URL>
1194,2022-08-28 23:59:57+09:00,1563904221845409792,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失礼だとまで思ってる。\n…ウソですほんとは超やりたいです。,84337504,Twitter Web App,鳥酋長,bird_chief,0,0,0,0,False,False,False,ja,[],https://twitter.com/84337504/status/1563904221845409794,スプラトゥーン３はあきらめる。\nあのゲームをSwitchライトでやるのはきつい。というか失礼だとまで思ってる。\n…ウソですほんとは超やりたいです。


In [20]:
# 最後に言語からのフィルタを考える
# 中国語と日本語は一部の漢字を共有しているため、収集時のクエリに漢字が含まれる場合は除いておく
original_tweets.query("lang != 'ja'").loc[:, ["name", "lang", "text"]]

,name,lang,text
190,re inbow,en,https://t.co/kGuwbxc0d8 T.A.T.U - All the things she said ( live 2013 Russia) ＃ニセモノ #やらせ #フェイク #standwithrussia #ティプシー #1d #inizon
207,re inbow,en,https://t.co/9KYSoYBV8q t.A.T.u. — «Show Me Love ＃ニセモノ #やらせ #フェイク #standwithrussia #ティプシー #1d #inizon
494,答应我不要做别人的猫好吗我会哭的,zh,嘘嘘做不了背头是不是因为秃
217,白愁飞的BOT,zh,将欲取天下而为之，吾见其不得已。天下神器，不可为也，不可执也。为者败之，执者失之。是以圣人无为，故无败；无执，故无失。\n\n夫物或行或随；或嘘或吹；或强或羸；或载或隳。\n\n是以圣人去甚，去奢，去泰。
722,盖亚！,zh,@Shinijuanjie 这是能说的嘛 嘘🤫
...,...,...,...
1173,傲娇娘🃏丝茉茉,zh,@mingysakra 要买个能长时间带的锁 不然每次嘘嘘满地撒
922,相模原のM(8/30J3第20節延期分vs福島),zh,#炎上覚悟で嫌いなものを言う\n讀賣嘘塵軍 https://t.co/IPLj5fUnNC
1036,pic,qht,#嘘
797,hanaMiiiii🧋,zh,看完最新漫畫：kdj，全部男人都愛你（嘘


# まとめ

- RTはツイート全体に対してかなり大きな割合を占めるため最初に考える
- sourceを見ることで機械的な投稿の影響を減らすことができる
- 収集時のクエリによっては部分一致や言語の影響があるため、ある程度機械的に除くことを考える

いずれも、「検証したい仮説への影響」と「省ける作業コスト」との天秤になる  
迷ったらゼミで上の2つを見せた上で議論すると、よりよいクリーニングの方法が見つかるかもしれない
また、今回使ったwi